In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Set the path to your dataset
data_path = 'path/to/your/pneumonia_dataset/'

# Load metadata containing image paths and labels
metadata = pd.read_csv(os.path.join(data_path, 'metadata.csv'))

# Encode labels using LabelEncoder
label_encoder = LabelEncoder()
metadata['encoded_labels'] = label_encoder.fit_transform(metadata['Label'])

# Split the data into training and testing sets
train_metadata, test_metadata = train_test_split(metadata, test_size=0.2, random_state=42, stratify=metadata['encoded_labels'])

# Create ImageDataGenerators for training and testing
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow data from the directory and apply data augmentation
train_generator = train_datagen.flow_from_dataframe(dataframe=train_metadata, directory=data_path,
                                                    x_col='filename', y_col='Label', target_size=(150, 150),
                                                    class_mode='categorical', batch_size=32, shuffle=True)

test_generator = test_datagen.flow_from_dataframe(dataframe=test_metadata, directory=data_path,
                                                  x_col='filename', y_col='Label', target_size=(150, 150),
                                                  class_mode='categorical', batch_size=32, shuffle=False)

# Build a CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(2, activation='softmax')  # Assuming binary classification (pneumonia or not)
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks (optional but recommended)
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Train the model
history = model.fit(train_generator, epochs=10, validation_data=test_generator, callbacks=[early_stopping, model_checkpoint])

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc}')

# Save the model
model.save('pneumonia_model.h5')


ReadError: ignored